In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
import os

TumorPath = ".\\Tumor\\Brain Tumor"
HealthyPath = ".\\Tumor\\Healthy"
X = []

for r,ds,fs in os.walk(TumorPath):
    break
for image in fs:
    img = cv2.imread(TumorPath+"\\"+image)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if img.shape[0] != 128 or img.shape[1] != 128:
        img = cv2.resize(img, dsize=(128,128))
    X.append(img)
y1 = np.ones((len(X),))
lastXlen = len(X)
for r,ds,fs in os.walk(HealthyPath):
    break
for image in fs:
    img = cv2.imread(HealthyPath+"\\"+image)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if img.shape[0] != 128 or img.shape[1] != 128:
        img = cv2.resize(img, dsize=(128,128))
    X.append(img)
y2 = np.zeros((len(X)-lastXlen,))
X = np.array(X, dtype=np.float32)
y = np.concat([y1,y2])
print(X.shape)
print(y.shape)

(4606, 128, 128)
(4606,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train = X_train.reshape(X_train.shape[0],128,128,1)
X_test = X_test.reshape(X_test.shape[0],128,128,1)

print("اندازه داده‌ها:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

اندازه داده‌ها:
X_train: (3684, 128, 128, 1)
X_test: (922, 128, 128, 1)
y_train: (3684,)
y_test: (922,)


In [5]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

output_activation = 'sigmoid'
loss_fn = 'binary_crossentropy'

# ساخت مدل
model = Sequential([
    # Block 1
    Conv2D(8, (3,3), activation='relu', padding='same', input_shape=(128,128,1)),
    BatchNormalization(),
    Conv2D(8, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Block 2
    Conv2D(16, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(16, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Block 3
    Conv2D(24, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(24, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Flatten + Dense layers
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation=output_activation)
])

# کامپایل مدل
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

# خلاصه مدل
model.summary()

# آموزش مدل
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=32
)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 8)    │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 128, 128, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 8)    │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 128, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 24)     │         3,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 32, 32, 24)     │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 24)     │         5,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32, 32, 24)     │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6144)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       393,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 406,569 (1.55 MB)

 Trainable params: 406,377 (1.55 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 15s 106ms/step - accuracy: 0.7614 - loss: 0.5382 - val_accuracy: 0.7983 - val_loss: 0.4325
Epoch 2/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 13s 113ms/step - accuracy: 0.8561 - loss: 0.3338 - val_accuracy: 0.8785 - val_loss: 0.2918
Epoch 3/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 118ms/step - accuracy: 0.8917 - loss: 0.2426 - val_accuracy: 0.9121 - val_loss: 0.2231
Epoch 4/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.9188 - loss: 0.1788 - val_accuracy: 0.9425 - val_loss: 0.1546
Epoch 5/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 118ms/step - accuracy: 0.9465 - loss: 0.1265 - val_accuracy: 0.9458 - val_loss: 0.1610
Epoch 6/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.9354 - loss: 0.1655 - val_accuracy: 0.9436 - val_loss: 0.1577
Epoch 7/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 118ms/step - accuracy: 0.9444 - loss: 0.1361 - val_accuracy: 0.9393 - val_loss: 0.1865
Epoch 8/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.9568 - loss: 0

In [6]:
model.save('Tumor_VGG.keras')